In [1]:
# Cell 1 – Imports & path setup
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_theme(style="whitegrid")

PROJ_ROOT = Path.cwd()
while PROJ_ROOT != PROJ_ROOT.parent and not (PROJ_ROOT / "data").exists():
    PROJ_ROOT = PROJ_ROOT.parent

DATA_DIR = PROJ_ROOT / "data" / "processed"
REPORTS_DIR = PROJ_ROOT / "reports"
TABLES_DIR = REPORTS_DIR / "tables"
FIGURES_DIR = REPORTS_DIR / "figures"

for path in [REPORTS_DIR, TABLES_DIR, FIGURES_DIR]:
    path.mkdir(parents=True, exist_ok=True)

PARQUET_PATH = DATA_DIR / "events.parquet"
CSV_PATH = DATA_DIR / "clean_data.csv"


In [2]:
# Cell 2 – Load processed dataset
if PARQUET_PATH.exists():
    df = pd.read_parquet(PARQUET_PATH)
    source_path = PARQUET_PATH
else:
    df = pd.read_csv(CSV_PATH, low_memory=False)
    source_path = CSV_PATH

print(f"Loaded {len(df):,} rows from {source_path}")
df.head()


Loaded 90,189 rows from c:\Users\umyana\Documents\mobile_game_analytics_pipeline\data\processed\events.parquet


,userid,version,session_count,retention_1,retention_7,acquisition_channel,country,platform,purchase,CAC,revenue,ROI
0,116,gate_30,3,0,0,Facebook,USA,Google Play,0,2.8,0.023765,-0.991512
1,337,gate_30,38,0,0,TikTok,USA,Google Play,0,1.7,0.062804,-0.963057
2,377,gate_40,165,1,0,Facebook,USA,Google Play,0,2.8,0.087634,-0.968702
3,483,gate_40,1,0,0,Facebook,Mexico,Google Play,0,2.8,0.011883,-0.995756
4,488,gate_40,179,0,1,TikTok,USA,App Store,0,1.7,0.771525,-0.546162


In [3]:
# Cell 3 – Sanity checks
required_cols = {
    "userid",
    "acquisition_channel",
    "platform",
    "version",
    "retention_1",
    "retention_7",
}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {sorted(missing)}")

summary = df[["retention_1", "retention_7"]].mean().rename("rate")
print(f"Average D1 retention: {summary['retention_1']:.2%}")
print(f"Average D7 retention: {summary['retention_7']:.2%}")


Average D1 retention: 45.49%
Average D7 retention: 33.29%


In [4]:
# Cell 4 – Retention by acquisition channel
channel_retention = (
    df.groupby("acquisition_channel", as_index=False)
    .agg(
        users=("userid", "nunique"),
        d1_rate=("retention_1", "mean"),
        d7_rate=("retention_7", "mean"),
    )
)

channel_retention["d7_from_d1"] = channel_retention["d7_rate"] / channel_retention["d1_rate"].replace(0, np.nan)
channel_retention = channel_retention.sort_values("d7_rate", ascending=False)

channel_retention


,acquisition_channel,users,d1_rate,d7_rate,d7_from_d1
2,Organic,9153,0.476674,0.351906,0.738253
1,Instagram,36281,0.459855,0.337697,0.734356
0,Facebook,26776,0.448947,0.327084,0.728558
3,TikTok,17979,0.442516,0.322432,0.728632


In [5]:
# Cell 5 – Export channel retention table
retention_csv_path = TABLES_DIR / "retention_by_channel.csv"
channel_retention.round(6).to_csv(retention_csv_path, index=False)
print(f"Saved channel retention table to {retention_csv_path}")


Saved channel retention table to c:\Users\umyana\Documents\mobile_game_analytics_pipeline\reports\tables\retention_by_channel.csv


In [6]:
# Cell 6 – Cohort table by version
cohort_version = (
    df.groupby(["version", "acquisition_channel"], as_index=False)
    .agg(
        users=("userid", "nunique"),
        d1_rate=("retention_1", "mean"),
        d7_rate=("retention_7", "mean"),
    )
)

cohort_version_path = TABLES_DIR / "retention_cohort_by_version.csv"
cohort_version.round(6).to_csv(cohort_version_path, index=False)
print(f"Saved version-level cohort table to {cohort_version_path}")
cohort_version.head()


Saved version-level cohort table to c:\Users\umyana\Documents\mobile_game_analytics_pipeline\reports\tables\retention_cohort_by_version.csv


,version,acquisition_channel,users,d1_rate,d7_rate
0,gate_30,Facebook,13288,0.452288,0.330975
1,gate_30,Instagram,17960,0.466704,0.339866
2,gate_30,Organic,4572,0.477034,0.358268
3,gate_30,TikTok,8880,0.447297,0.324550
4,gate_40,Facebook,13488,0.445655,0.323250


In [7]:
# Cell 7 – Heatmap for D7 retention
heatmap_data = cohort_version.pivot_table(
    index="acquisition_channel",
    columns="version",
    values="d7_rate",
)

plt.figure(figsize=(10, 4 + len(heatmap_data) * 0.3))
sns.heatmap(
    heatmap_data,
    cmap="YlGnBu",
    annot=True,
    fmt=".1%",
    cbar_kws={"label": "D7 retention"},
)

plt.title("D7 Retention by Version and Channel")
plt.xlabel("Version")
plt.ylabel("Acquisition channel")
plt.tight_layout()

heatmap_path = FIGURES_DIR / "retention_heatmap.png"
plt.savefig(heatmap_path, dpi=150, bbox_inches="tight")
plt.close()
print(f"Saved heatmap to {heatmap_path}")


Saved heatmap to c:\Users\umyana\Documents\mobile_game_analytics_pipeline\reports\figures\retention_heatmap.png


In [8]:
# Cell 8 – Text summary
top_d7 = channel_retention.iloc[0]
worst_d7 = channel_retention.iloc[-1]

print(
    f"Highest D7 channel: {top_d7['acquisition_channel']} -> {top_d7['d7_rate']:.2%}. "
    f"Lowest D7 channel: {worst_d7['acquisition_channel']} -> {worst_d7['d7_rate']:.2%}."
)


Highest D7 channel: Organic -> 35.19%. Lowest D7 channel: TikTok -> 32.24%.
